In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd

import numpy as np

from dcgan.components.generator import Generator
from dcgan.components.discriminator import Discriminator

In [2]:
def fisher(likelihood, params):
    # real_label=torch.full((batch_size, ), 1, dtype=torch.float)
    # l_adv = criterion(output, real_label)
    
    grad1 = autograd.grad(likelihood, params, create_graph=True)
    fisher = []

    for i, g in enumerate(grad1):
        print(i)
        # maybe fisher is first deriv?
        # grad2 = torch.autograd.grad(g, gen, grad_outputs=torch.ones_like(g), retain_graph= True)
        fisher.append((g**2).sum()) 

    return fisher

# def loss_adapt(output, gen_param, label):
#     return nn.BCELoss(output, label) + regWeight*Fisher(output, gen_param)

In [3]:
generator = Generator()
discriminator = Discriminator()

generator.load_state_dict(torch.load('models/epoch145_gen.pth'))
discriminator.load_state_dict(torch.load('models/epoch145_dis.pth'))

<All keys matched successfully>

In [4]:
batch_size = 256
latent_size = 512
regWeight = 10
criterion = nn.BCELoss()

noise = torch.randn(batch_size, latent_size)
output = discriminator(generator(noise)).view(-1)
real_label=torch.full((batch_size, ), 1, dtype=torch.float)
l_adv = criterion(output, real_label)

fisher = fisher(l_adv, generator.parameters())

# loss_adapt(discriminator(generator(noise)).view(-1), generator.parameters(), 1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [7]:
import matplotlib.pyplot as plt

fisher_vals = [t.item() for t in fisher]

plt.bar(np.arange(len(fisher_vals)), fisher_vals)
plt.show()